# Summary

The [Yelp Dataset on Kaggle](https://www.kaggle.com/yelp-dataset/yelp-dataset) has been downloaded
and given a data audit. Each data file has been combined to create a single larger dataset.

# Load Dependencies

In [1]:
import os

from bokeh.io import show, output_notebook
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, TapTool, BoxSelectTool
from bokeh.models.graphs import from_networkx, NodesAndLinkedEdges, EdgesAndLinkedNodes
from bokeh.palettes import Spectral4
from IPython.display import display_html
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import numpy as np

%matplotlib inline

# Helper Functions

In [2]:
def get_edge_data(G, key):
    ok = []
    for i, o, d in G.edges(data=True):
        ok.append(d[key])
    return ok

def display_side_by_side(dfs: list):
    """ Reference: https://stackoverflow.com/questions/38783027 """
    html_str=''
    for df in dfs:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
def construct_side_by_side(df_sum: pd.Series, segments: int, names: list):
    """ Create dataframe summary, then use this function for construction. """
    prev = 0
    dfs = []
    for i in range(0,df_sum.shape[0], segments):
        if i != 0:
            sub = pd.DataFrame(df_sum.iloc[prev:i].reset_index())
            sub.columns = names
            dfs.append(sub)
            prev = i

    sub = pd.DataFrame(df_sum.iloc[i:].reset_index())
    sub.columns = names
    dfs.append(sub)
    return dfs

# Load Data

In [3]:
yelp_business = pd.read_csv("data/yelp_business.csv")
yelp_business_attrs = pd.read_csv("data/yelp_business_attributes.csv")
yelp_business_hrs = pd.read_csv("data/yelp_business_hours.csv")
yelp_checkin = pd.read_csv("data/yelp_checkin.csv")
yelp_review = pd.read_csv("data/yelp_review.csv")
yelp_tip = pd.read_csv("data/yelp_tip.csv")
yelp_user = pd.read_csv("data/yelp_user.csv")

# Data Relationships

I've graphed the relationships between each data file 
using the existence of a foreign key to define edges. This
is an interactive figure where hovering your mouse over 
an edge provides relevant information. This graph can be
helpful when deciding if it makes sense to recombine the
data differently at some point.

In [4]:
G = nx.Graph()
#G = nx.DiGraph()
DIR_LOC = "data"

In [5]:
fnames = [i for i in os.listdir(DIR_LOC) if i[-4:] == ".csv"]
fnames.sort()
fnames

['yelp_business.csv',
 'yelp_business_attributes.csv',
 'yelp_business_hours.csv',
 'yelp_checkin.csv',
 'yelp_review.csv',
 'yelp_tip.csv',
 'yelp_user.csv']

In [6]:
G.add_nodes_from(fnames)

In [7]:
G.add_edge('yelp_business.csv', 'yelp_business_attributes.csv')
G['yelp_business.csv']['yelp_business_attributes.csv']['foreign_key'] = "business_id"

G.add_edge('yelp_business.csv', 'yelp_business_hours.csv')
G.add_edge('yelp_business_attributes.csv', 'yelp_business_hours.csv')
G['yelp_business.csv']['yelp_business_hours.csv']['foreign_key'] = "business_id"
G['yelp_business_attributes.csv']['yelp_business_hours.csv']['foreign_key'] = "business_id"

G.add_edge('yelp_business.csv', 'yelp_checkin.csv')
G.add_edge('yelp_business_hours.csv', 'yelp_checkin.csv')
G.add_edge('yelp_business_attributes.csv', 'yelp_checkin.csv')
G['yelp_business.csv']['yelp_checkin.csv']['foreign_key'] = "business_id"
G['yelp_business_hours.csv']['yelp_checkin.csv']['foreign_key'] = "business_id"
G['yelp_business_attributes.csv']['yelp_checkin.csv']['foreign_key'] = "business_id"

G.add_edge('yelp_review.csv', 'yelp_business.csv')
G.add_edge('yelp_review.csv', 'yelp_business_hours.csv')
G.add_edge('yelp_review.csv', 'yelp_business_attributes.csv')
G.add_edge('yelp_review.csv', 'yelp_checkin.csv')
G['yelp_review.csv']['yelp_business.csv']['foreign_key'] = "business_id"
G['yelp_review.csv']['yelp_business_hours.csv']['foreign_key'] = "business_id"
G['yelp_review.csv']['yelp_business_attributes.csv']['foreign_key'] = "business_id"
G['yelp_review.csv']['yelp_checkin.csv']['foreign_key'] = "business_id"

G.add_edge('yelp_review.csv', 'yelp_tip.csv')
G.add_edge('yelp_review.csv', 'yelp_user.csv')
G['yelp_review.csv']['yelp_tip.csv']['foreign_key'] = "user_id"
G['yelp_review.csv']['yelp_user.csv']['foreign_key'] = "user_id"


In [8]:
TOOL_TIPS = [
    ("Edge", "@edgename"),
    ("Foreign Key", "@attr"),
    
]

plot = Plot(plot_width=600, plot_height=600,
            x_range=Range1d(-1.1,1.1), y_range=Range1d(-1.1,1.1))
plot.title.text = "Yelp Data Schema"

plot.add_tools(HoverTool(tooltips=TOOL_TIPS), TapTool(), BoxSelectTool())

graph_renderer = from_networkx(G, nx.circular_layout, scale=1, center=(0,0))

#graph_renderer.node_renderer.data_source.data['wutang'] = list(G.edges())
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color=Spectral4[0])
graph_renderer.node_renderer.selection_glyph = Circle(size=15, fill_color=Spectral4[2])
graph_renderer.node_renderer.hover_glyph = Circle(size=15, fill_color=Spectral4[1])

graph_renderer.edge_renderer.glyph = MultiLine(line_color="#CCCCCC", line_alpha=0.8, line_width=5)
graph_renderer.edge_renderer.selection_glyph = MultiLine(line_color=Spectral4[2], line_width=5)
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color=Spectral4[1], line_width=5)
graph_renderer.edge_renderer.data_source.data['edgename'] = list(G.edges())
graph_renderer.edge_renderer.data_source.data['attr'] = get_edge_data(G, 'foreign_key')

graph_renderer.selection_policy = NodesAndLinkedEdges()
#graph_renderer.inspection_policy = NodesAndLinkedEdges()
graph_renderer.inspection_policy = EdgesAndLinkedNodes()

plot.renderers.append(graph_renderer)

#output_file("interactive_graphs.html")
output_notebook()
show(plot)

Loading BokehJS ...

# Data Audit

We want to understand the number of zero values and/or NA values 
per column. This will inform the efficacy of the model later in
the process. It's important to know how much usable information
is in each feature. The data audit consists of the following:

* Percent of NA values per attribute
* Percent of zero values per attribute
* Percent of NA and zero values per attribute

## yelp_business.csv

In [9]:
yelp_business.shape

(174567, 13)

In [10]:
yelp_business.columns

Index(['business_id', 'name', 'neighborhood', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars', 'review_count',
       'is_open', 'categories'],
      dtype='object')

In [11]:
yelp_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [12]:
# Percent of NA values per attribute

na_sum = round((yelp_business.isna().sum() / yelp_business.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=5, names=["attr", "NA %"]))

,attr,NA %
0,neighborhood,61.04
1,postal_code,0.36
2,categories,0.00
3,is_open,0.00
4,review_count,0.00
,attr,NA %
0,stars,0.0
1,longitude,0.0
2,latitude,0.0
3,state,0.0


In [13]:
# Percent of zero values per attribute

zero = yelp_business.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_business.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=5, names=["attr", "Zero val %"]))

,attr,Zero val %
0,is_open,15.96
1,categories,0.00
2,review_count,0.00
3,stars,0.00
4,longitude,0.00
,attr,Zero val %
0,latitude,0.0
1,postal_code,0.0
2,state,0.0
3,city,0.0


In [14]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=5, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,neighborhood,61.04
1,is_open,15.96
2,postal_code,0.36
3,state,0.00
4,stars,0.00
,attr,Zero/NA val %
0,review_count,0.0
1,name,0.0
2,longitude,0.0
3,latitude,0.0


## yelp_business_attrs.csv

In [15]:
yelp_business_attrs.shape

(152041, 82)

In [16]:
yelp_business_attrs.columns

Index(['business_id', 'AcceptsInsurance', 'ByAppointmentOnly',
       'BusinessAcceptsCreditCards', 'BusinessParking_garage',
       'BusinessParking_street', 'BusinessParking_validated',
       'BusinessParking_lot', 'BusinessParking_valet',
       'HairSpecializesIn_coloring', 'HairSpecializesIn_africanamerican',
       'HairSpecializesIn_curly', 'HairSpecializesIn_perms',
       'HairSpecializesIn_kids', 'HairSpecializesIn_extensions',
       'HairSpecializesIn_asian', 'HairSpecializesIn_straightperms',
       'RestaurantsPriceRange2', 'GoodForKids', 'WheelchairAccessible',
       'BikeParking', 'Alcohol', 'HasTV', 'NoiseLevel', 'RestaurantsAttire',
       'Music_dj', 'Music_background_music', 'Music_no_music', 'Music_karaoke',
       'Music_live', 'Music_video', 'Music_jukebox', 'Ambience_romantic',
       'Ambience_intimate', 'Ambience_classy', 'Ambience_hipster',
       'Ambience_divey', 'Ambience_touristy', 'Ambience_trendy',
       'Ambience_upscale', 'Ambience_casual', 'Restau

In [17]:
yelp_business_attrs.head()

,business_id,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,...,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,FYWN1wneV18bWNgQjJ2GNg,Na,Na,Na,True,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
1,He-G7vWjzVUysIKrfNbPUQ,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
2,8DShNS-LuFqpEWIp0HxijA,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
3,PfOCPjBrlQAnz__NXj9h_w,Na,Na,Na,Na,Na,Na,Na,Na,Na,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na
4,o9eMRCWt5PkpLDE0gOPtcQ,Na,Na,Na,Na,False,False,False,False,False,...,Na,Na,Na,Na,Na,Na,Na,Na,Na,Na


In [18]:
yelp_business_attrs[yelp_business_attrs == 'Na'] = np.NaN
yelp_business_attrs.head()

,business_id,AcceptsInsurance,ByAppointmentOnly,BusinessAcceptsCreditCards,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,HairSpecializesIn_coloring,...,Corkage,DietaryRestrictions_dairy-free,DietaryRestrictions_gluten-free,DietaryRestrictions_vegan,DietaryRestrictions_kosher,DietaryRestrictions_halal,DietaryRestrictions_soy-free,DietaryRestrictions_vegetarian,AgesAllowed,RestaurantsCounterService
0,FYWN1wneV18bWNgQjJ2GNg,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,He-G7vWjzVUysIKrfNbPUQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8DShNS-LuFqpEWIp0HxijA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,PfOCPjBrlQAnz__NXj9h_w,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,o9eMRCWt5PkpLDE0gOPtcQ,NaN,NaN,NaN,NaN,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# Percent of NA values per attribute

na_sum = round((yelp_business_attrs.isna().sum() / yelp_business_attrs.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=10, names=["attr", "NA %"]))

,attr,NA %
0,RestaurantsCounterService,100.0
1,AcceptsInsurance,100.0
2,HairSpecializesIn_africanamerican,100.0
3,HairSpecializesIn_curly,100.0
4,HairSpecializesIn_perms,100.0
5,HairSpecializesIn_kids,100.0
6,HairSpecializesIn_extensions,100.0
7,DietaryRestrictions_dairy-free,100.0
8,Corkage,100.0
9,HairSpecializesIn_asian,100.0


In [20]:
# Percent of zero values per attribute

zero = yelp_business_attrs.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_business_attrs.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=10, names=["attr", "Zero val %"]))

,attr,Zero val %
0,RestaurantsCounterService,0.0
1,BikeParking,0.0
2,HasTV,0.0
3,NoiseLevel,0.0
4,RestaurantsAttire,0.0
5,Music_dj,0.0
6,Music_background_music,0.0
7,Music_no_music,0.0
8,Music_karaoke,0.0
9,Music_live,0.0


In [21]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=10, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,AcceptsInsurance,100.0
1,HairSpecializesIn_straightperms,100.0
2,Corkage,100.0
3,DietaryRestrictions_dairy-free,100.0
4,HairSpecializesIn_africanamerican,100.0
5,HairSpecializesIn_asian,100.0
6,HairSpecializesIn_curly,100.0
7,HairSpecializesIn_extensions,100.0
8,HairSpecializesIn_kids,100.0
9,HairSpecializesIn_perms,100.0


## yelp_business_hours.csv

In [22]:
yelp_business_hrs.shape

(174567, 8)

In [23]:
yelp_business_hrs.columns

Index(['business_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday',
       'saturday', 'sunday'],
      dtype='object')

In [24]:
yelp_business_hrs.head()

,business_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,FYWN1wneV18bWNgQjJ2GNg,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,7:30-17:0,None,None
1,He-G7vWjzVUysIKrfNbPUQ,9:0-20:0,9:0-20:0,9:0-20:0,9:0-20:0,9:0-16:0,8:0-16:0,None
2,KQPW8lFf1y5BT2MxiSZ3QA,None,None,None,None,None,None,None
3,8DShNS-LuFqpEWIp0HxijA,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,10:0-21:0,11:0-19:0
4,PfOCPjBrlQAnz__NXj9h_w,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-1:0,11:0-2:0,11:0-0:0


In [25]:
yelp_business_hrs[yelp_business_hrs == 'None'] = np.NaN

In [26]:
# Percent of NA values per attribute

na_sum = round((yelp_business_hrs.isna().sum() / yelp_business_hrs.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=5, names=["attr", "NA %"]))

,attr,NA %
0,sunday,52.09
1,saturday,35.25
2,monday,31.90
3,tuesday,27.67
4,friday,27.00
,attr,NA %
0,wednesday,27.00
1,thursday,26.71
2,business_id,0.00


In [27]:
# Percent of zero values per attribute

zero = yelp_business_hrs.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_business_hrs.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=5, names=["attr", "Zero val %"]))

,attr,Zero val %
0,sunday,0.0
1,saturday,0.0
2,friday,0.0
3,thursday,0.0
4,wednesday,0.0
,attr,Zero val %
0,tuesday,0.0
1,monday,0.0
2,business_id,0.0


In [28]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=5, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,sunday,52.09
1,saturday,35.25
2,monday,31.90
3,tuesday,27.67
4,wednesday,27.00
,attr,Zero/NA val %
0,friday,27.00
1,thursday,26.71
2,business_id,0.00


## yelp_checkin.csv

In [29]:
yelp_checkin.shape

(3911218, 4)

In [30]:
yelp_checkin.columns

Index(['business_id', 'weekday', 'hour', 'checkins'], dtype='object')

In [31]:
yelp_checkin.head()

,business_id,weekday,hour,checkins
0,3Mc-LxcqeguOXOVT_2ZtCg,Tue,0:00,12
1,SVFx6_epO22bZTZnKwlX7g,Wed,0:00,4
2,vW9aLivd4-IorAfStzsHww,Tue,14:00,1
3,tEzxhauTQddACyqdJ0OPEQ,Fri,19:00,1
4,CEyZU32P-vtMhgqRCaXzMA,Tue,17:00,1


In [32]:
# Percent of NA values per attribute

na_sum = round((yelp_checkin.isna().sum() / yelp_checkin.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=5, names=["attr", "NA %"]))

,attr,NA %
0,checkins,0.0
1,hour,0.0
2,weekday,0.0
3,business_id,0.0


In [33]:
# Percent of zero values per attribute

zero = yelp_checkin.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_checkin.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=5, names=["attr", "Zero val %"]))

,attr,Zero val %
0,checkins,0.0
1,hour,0.0
2,weekday,0.0
3,business_id,0.0


In [34]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=5, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,business_id,0.0
1,weekday,0.0
2,hour,0.0
3,checkins,0.0


## yelp_review.csv

In [35]:
yelp_review.shape

(5261668, 9)

In [36]:
yelp_review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'date', 'text',
       'useful', 'funny', 'cool'],
      dtype='object')

In [37]:
yelp_review.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [38]:
# Percent of NA values per attribute

na_sum = round((yelp_review.isna().sum() / yelp_review.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=5, names=["attr", "NA %"]))

,attr,NA %
0,cool,0.0
1,funny,0.0
2,useful,0.0
3,text,0.0
4,date,0.0
,attr,NA %
0,stars,0.0
1,business_id,0.0
2,user_id,0.0
3,review_id,0.0


In [39]:
# Percent of zero values per attribute

zero = yelp_review.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_review.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=5, names=["attr", "Zero val %"]))

,attr,Zero val %
0,funny,78.51
1,cool,74.03
2,useful,52.16
3,text,0.00
4,date,0.00
,attr,Zero val %
0,stars,0.0
1,business_id,0.0
2,user_id,0.0
3,review_id,0.0


In [40]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=5, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,funny,78.51
1,cool,74.03
2,useful,52.16
3,user_id,0.00
4,text,0.00
,attr,Zero/NA val %
0,stars,0.0
1,review_id,0.0
2,date,0.0
3,business_id,0.0


## yelp_tip.csv

In [41]:
yelp_tip.shape

(1098324, 5)

In [42]:
yelp_tip.columns

Index(['text', 'date', 'likes', 'business_id', 'user_id'], dtype='object')

In [43]:
yelp_tip.head()

,text,date,likes,business_id,user_id
0,Great breakfast large portions and friendly wa...,2015-08-12,0,jH19V2I9fIslnNhDzPmdkA,ZcLKXikTHYOnYt5VYRO5sg
1,Nice place. Great staff. A fixture in the tow...,2014-06-20,0,dAa0hB2yrnHzVmsCkN4YvQ,oaYhjqBbh18ZhU0bpyzSuw
2,Happy hour 5-7 Monday - Friday,2016-10-12,0,dAa0hB2yrnHzVmsCkN4YvQ,ulQ8Nyj7jCUR8M83SUMoRQ
3,"Parking is a premium, keep circling, you will ...",2017-01-28,0,ESzO3Av0b1_TzKOiqzbQYQ,ulQ8Nyj7jCUR8M83SUMoRQ
4,Homemade pasta is the best in the area,2017-02-25,0,k7WRPbDd7rztjHcGGkEjlw,ulQ8Nyj7jCUR8M83SUMoRQ


In [44]:
# Percent of NA values per attribute

na_sum = round((yelp_tip.isna().sum() / yelp_tip.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=5, names=["attr", "NA %"]))

,attr,NA %
0,user_id,0.0
1,business_id,0.0
2,likes,0.0
3,date,0.0
4,text,0.0


In [45]:
# Percent of zero values per attribute

zero = yelp_tip.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_tip.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=5, names=["attr", "Zero val %"]))

,attr,Zero val %
0,likes,98.52
1,user_id,0.00
2,business_id,0.00
3,date,0.00
4,text,0.00


In [46]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=5, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,likes,98.52
1,user_id,0.00
2,text,0.00
3,date,0.00
4,business_id,0.00


## yelp_user.csv

In [47]:
yelp_user.shape

(1326100, 22)

In [48]:
yelp_user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'friends', 'useful',
       'funny', 'cool', 'fans', 'elite', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [49]:
yelp_user.head()

,user_id,name,review_count,yelping_since,friends,useful,funny,cool,fans,elite,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gpg...",0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
1,uUzsFQn_6cXDh6rPNGbIFA,Tiffy,1,2017-03-02,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
2,mBneaEEH5EMyxaVyqS-72A,Mark,6,2015-03-13,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
3,W5mJGs-dcDWRGEhAzUYtoA,Evelyn,3,2016-09-08,None,0,0,0,0,None,...,0,0,0,0,0,0,0,0,0,0
4,4E8--zUZO1Rr1IBK4_83fg,Lisa,11,2012-07-16,None,4,0,0,0,None,...,0,0,0,0,0,0,0,0,1,0


In [50]:
yelp_user[yelp_user == 'None'] = np.NaN

In [51]:
# Percent of NA values per attribute

na_sum = round((yelp_user.isna().sum() / yelp_user.shape[0]) * 100,2)
na_sum = na_sum.sort_values(ascending=False)
display_side_by_side(construct_side_by_side(na_sum, segments=8, names=["attr", "NA %"]))

,attr,NA %
0,elite,95.41
1,friends,42.69
2,name,0.04
3,compliment_photos,0.00
4,compliment_writer,0.00
5,review_count,0.00
6,yelping_since,0.00
7,useful,0.00
,attr,NA %
0,funny,0.0


In [52]:
# Percent of zero values per attribute

zero = yelp_user.applymap(lambda x: True if x == 0.0 else False)
zero_sum = round((zero.sum() / yelp_user.shape[0]) * 100, 2)
zero_sum = zero_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_sum, segments=8, names=["attr", "Zero val %"]))

,attr,Zero val %
0,compliment_list,98.60
1,compliment_cute,96.79
2,compliment_profile,96.40
3,compliment_photos,93.63
4,compliment_more,91.56
5,compliment_hot,90.14
6,compliment_writer,89.05
7,compliment_cool,86.49
,attr,Zero val %
0,compliment_funny,86.49


In [53]:
# Percent of zero or NA values per attribute

zero_or_na_sum = zero_sum + na_sum
zero_or_na_sum = zero_or_na_sum.sort_values(ascending=False)

display_side_by_side(construct_side_by_side(zero_or_na_sum, segments=8, names=["attr", "Zero/NA val %"]))

,attr,Zero/NA val %
0,compliment_list,98.60
1,compliment_cute,96.79
2,compliment_profile,96.40
3,elite,95.41
4,compliment_photos,93.63
5,compliment_more,91.56
6,compliment_hot,90.14
7,compliment_writer,89.05
,attr,Zero/NA val %
0,compliment_cool,86.49


# Merging Data

Given what we know from the data audit, let's merge the dataset.

# Useful subsets

Remove any column which is not more than 70% full of values besides 0 or NA. This
approach will be incorrect if a variable is categorical and the default class is
zero. Modifications will be necessary if this happens.

In [54]:
yelp_business = yelp_business[[i for i in yelp_business.columns if i != 'neighborhood']]
yelp_business.shape

(174567, 12)

In [55]:
# yelp_business_attrs; disregard

In [56]:
# keep all
yelp_business_hrs.shape

(174567, 8)

In [57]:
# keep all
yelp_checkin.shape

(3911218, 4)

In [58]:
yelp_review = yelp_review[[i for i in yelp_review.columns if i not in ['funny','cool']]]
yelp_review.shape

(5261668, 7)

In [59]:
yelp_tip = yelp_tip[[i for i in yelp_tip.columns if i not in 'likes']]
yelp_tip.shape

(1098324, 4)

In [60]:
yelp_user = yelp_user[[i for i in yelp_user.columns if i in ['useful','friends','review_count','name',
                                                             'yelping_since','user_id','average_stars']]]
yelp_user.shape

(1326100, 7)

## Merge strategy.

We spoke about excludng features from text descriptions if possible. I'm going to
include them anyway. If those features don't work out. Here would be a good place
to remove them in the future. That way our data can be restructured upstream.

I'll do a left join to `yelp_review` since that dataset has the largest number
of instances. Substantively, it would make sense that the actual reviews make
up the most instances in the Yelp dataset.

In [61]:
#yelp_review.columns = ['review_id', 'user_id', 'business_id', 'stars_dependent', 'date', 'text', 'useful']

In [62]:
#df = pd.merge(yelp_review, yelp_user, how='left', on='user_id')
#df.shape

In [63]:
# Exclude many-to-many join for now

#df = pd.merge(df, yelp_tip, how='left', on='user_id')
#df.shape

In [64]:
#df = pd.merge(df, yelp_business, how='left', on='business_id')
#df.shape

In [65]:
#df = pd.merge(df, yelp_business_hrs, how='left', on='business_id')
#df.shape

## Dependent Variable

The dependent variable is `stars_dependent`. This variable is 
the number of stars given in a Yelp review.

In [67]:
#df.columns

In [68]:
#df.stars_dependent.describe()

In [69]:
#df.stars_dependent.hist()

Note that `yelp_business` and `yelp_review` have the overlapping `stars`
variable name.

In [70]:
#yelp_business.columns

In [71]:
#yelp_review.columns

# Output

CSV files have a tendency to get corrupted so I'll output in a couple formats.

Merging is not going to be possible due to file size. We'll have to think
of some other options.

In [72]:
#df.to_csv("data/yelp_df_cs6220.csv")